# Project 5 Person of Interest
## A detection and tracking system for public cameras

## Part I OpenCV tracker system comparison

## Compare trackers. 

Person of Interest: OpenCV Tracking API | TensorFlow | Deep learning | AWS
- Enable the camera watch for public security at New York Time Square.
- Developing a detection and tracking system for one of the public cameras.
- Setting notification of abnormal counts of people and their moving speed.

In [3]:
import cv2
import numpy as np
import gc
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
cv2.__version__

'3.3.0'

## I. Compared 6 trackers, decided the best one for this camera is MIL.

In [3]:
# try 6 trackers, BGR
#Boosting, MIL, KCF, TLD, MedianFlow, GOTURN
#https://github.com/opencv/opencv_contrib/blob/master/modules/tracking/samples/tracker.py

import numpy as np
import cv2
import sys

cv2.namedWindow("tracking")
camera = cv2.VideoCapture('./videos/0.mov')
ok, image=camera.read()
if not ok:
    print('Failed to read video')
    exit()

#bounding box, region of interest   
bbox = cv2.selectROI("tracking", image)
tracker_boosting = cv2.TrackerBoosting_create()
tracker_mil = cv2.TrackerMIL_create()
tracker_kcf = cv2.TrackerKCF_create() #lost and no faulse positive
tracker_tld = cv2.TrackerTLD_create() #very slow, lost in the middle then follow others
tracker_medianflow = cv2.TrackerMedianFlow_create() #lost in the middle, can shrink and expand to capture
# tracker_goturn = cv2.TrackerGOTURN_create() # not in this coding
init_once = False
i=0

while camera.isOpened():
    ok, image=camera.read()
    if not ok:
        print ('no image to read')
        break

    if not init_once:
        ok1 = tracker_boosting.init(image, bbox)
        ok2 = tracker_mil.init(image, bbox)
        ok3 = tracker_kcf.init(image, bbox)
        ok4 = tracker_tld.init(image, bbox)
        ok5 = tracker_medianflow.init(image, bbox)
#         ok = tracker_goturn.init(image, bbox)
        init_once = True

# for boosting box
    ok, newbox_boosting = tracker_boosting.update(image)
    if ok:
        p1 = (int(newbox_boosting[0]), int(newbox_boosting[1]))
        p2 = (int(newbox_boosting[0] + newbox_boosting[2]), int(newbox_boosting[1] + newbox_boosting[3]))
        cv2.rectangle(image, p1, p2, (200,0,0),2)
#         cv2.putText(image, 'boosting',p1,(100,100,0),4,cv2.LINE_AA)
#         print(p1)
 
# for mil
    ok, newbox_mil = tracker_mil.update(image)
    if ok:
        p1 = (int(newbox_mil[0]), int(newbox_mil[1]))
        p2 = (int(newbox_mil[0] + newbox_mil[2]), int(newbox_mil[1] + newbox_mil[3]))
        cv2.rectangle(image, p1, p2, (0,200,0),2)
        
# for kcf
    ok, newbox_kcf = tracker_kcf.update(image)
    if ok:
        p1 = (int(newbox_kcf[0]), int(newbox_kcf[1]))
        p2 = (int(newbox_kcf[0] + newbox_kcf[2]), int(newbox_kcf[1] + newbox_kcf[3]))
        cv2.rectangle(image, p1, p2, (0,0,200),2)

# for tld
    ok, newbox_tld = tracker_tld.update(image)
    if ok:
        p1 = (int(newbox_tld[0]), int(newbox_tld[1]))
        p2 = (int(newbox_tld[0] + newbox_tld[2]), int(newbox_tld[1] + newbox_tld[3]))
        cv2.rectangle(image, p1, p2, (0,0,0),2)

# for medianflow
    ok, newbox_medianflow = tracker_medianflow.update(image)
    if ok:
        p1 = (int(newbox_medianflow[0]), int(newbox_medianflow[1]))
        p2 = (int(newbox_medianflow[0] + newbox_medianflow[2]), int(newbox_medianflow[1] + newbox_medianflow[3]))
        cv2.rectangle(image, p1, p2, (200,200,200),2)

    cv2.imshow("tracking", image)
    cv2.imwrite( "../../../nyc17_ds12/projects/05-kojak/images/tracker/n14_1_%d.jpg" %i, image);
    i+=1
    k = cv2.waitKey(30) & 0xff
    if k == 27 : break # esc pressed
        
camera.release()
cv2.destroyAllWindows()
for i in range(1,5):
    cv2.waitKey(1)

## II. Multiple trackers enable the combination of tracker and object detection.
https://github.com/opencv/opencv_contrib/blob/master/modules/tracking/samples/multitracker.py

In [ ]:
import numpy as np
import cv2
import sys

if len(sys.argv) != 2:
    print('Input video name is missing')
    exit()

print('Select 3 tracking targets')

cv2.namedWindow("tracking")
camera = cv2.VideoCapture('./videos/0.mov')
tracker = cv2.TrackerGOTURN_create()
init_once = False

ok, image=camera.read()
if not ok:
    print('Failed to read video')
    exit()

bbox1 = cv2.selectROI('tracking', image)
bbox2 = cv2.selectROI('tracking', image)
bbox3 = cv2.selectROI('tracking', image)

while camera.isOpened():
    ok, image=camera.read()
    if not ok:
        print ('no image to read')
        break

    if not init_once:
        ok = tracker.add(cv2.TrackerMIL_create(), image, bbox1)
        ok = tracker.add(cv2.TrackerMIL_create(), image, bbox2)
        ok = tracker.add(cv2.TrackerMIL_create(), image, bbox3)
        init_once = True

    ok, boxes = tracker.update(image)
    print (ok, boxes)

    for newbox in boxes:
        p1 = (int(newbox[0]), int(newbox[1]))
        p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
        cv2.rectangle(image, p1, p2, (200,0,0))

    cv2.imshow('tracking', image)
    k = cv2.waitKey(1)
    if k == 27 : break # esc pressed
camera.release()
cv2.destroyAllWindows()
for i in range(1,5):
    cv2.waitKey(1)